In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.nn import GCNConv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2, size3):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(174, size1)
        self.conv2 = GCNConv(size1, size2)
        self.fc1 = torch.nn.Linear(size2, size3)
        self.fc2 = torch.nn.Linear(size3, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    size3 = trial.suggest_int("size3", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2, size3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [3]:
for rs in range(10):
    
    # read data
    scaler = MinMaxScaler()
    X_train = pd.read_csv("../../result/input_GTEx_phyloP/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_train = scaler.fit_transform(X_train)
    X_valid = pd.read_csv("../../result/input_GTEx_phyloP/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_valid = scaler.transform(X_valid)
    X_test = pd.read_csv("../../result/input_GTEx_phyloP/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    X_test = scaler.transform(X_test)
    Y_train = pd.read_csv("../../result/input_GTEx_phyloP/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_valid = pd.read_csv("../../result/input_GTEx_phyloP/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test = pd.read_csv("../../result/input_GTEx_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test_gene = pd.read_csv("../../result/input_GTEx_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_GTEx_phastcons/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_GTEx_phastcons/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_threshold0.3", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_GM12878", "CoExpr_ENCODE_GM12878_0.75", "CoExpr_GTEx_EBV_0.75", "Random_ER", "Random_SF"]:
        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_GTEx_phastcons/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_GTEx_phyloP/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"], best_params["size3"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_GTEx_phyloP/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-12-10 12:48:15,869] A new study created in memory with name: no-name-ed40ddfc-cd47-44ac-b440-cc60d454b5fd
[I 2023-12-10 12:48:19,514] Trial 0 finished with value: 0.2124735713005066 and parameters: {'size1': 592, 'size2': 405, 'size3': 323, 'num_epochs': 172, 'lr': 1.1240359262070523e-05}. Best is trial 0 with value: 0.2124735713005066.
[I 2023-12-10 12:48:20,114] Trial 1 finished with value: 0.14861156046390533 and parameters: {'size1': 226, 'size2': 230, 'size3': 128, 'num_epochs': 77, 'lr': 0.0008740347096341148}. Best is trial 1 with value: 0.14861156046390533.
[I 2023-12-10 12:48:21,336] Trial 2 finished with value: 0.16021916270256042 and parameters: {'size1': 143, 'size2': 307, 'size3': 277, 'num_epochs': 150, 'lr': 0.0014216217836480734}. Best is trial 1 with value: 0.14861156046390533.
[I 2023-12-10 12:48:23,011] Trial 3 finished with value: 0.21258413791656494 and parameters: {'size1': 527, 'size2': 470, 'size3': 335, 'num_epochs': 143, 'lr': 0.004200925021399221}. Be

[I 2023-12-10 12:48:48,597] Trial 12 pruned. 
[I 2023-12-10 12:48:48,997] Trial 13 finished with value: 0.15591932833194733 and parameters: {'size1': 130, 'size2': 197, 'size3': 30, 'num_epochs': 44, 'lr': 0.0034736824022015868}. Best is trial 13 with value: 0.15591932833194733.
[I 2023-12-10 12:48:49,067] Trial 14 pruned. 
[I 2023-12-10 12:48:49,130] Trial 15 pruned. 
[I 2023-12-10 12:48:49,219] Trial 16 pruned. 
[I 2023-12-10 12:48:49,632] Trial 17 finished with value: 0.1544594019651413 and parameters: {'size1': 40, 'size2': 255, 'size3': 19, 'num_epochs': 44, 'lr': 0.00999765291667031}. Best is trial 17 with value: 0.1544594019651413.
[I 2023-12-10 12:48:49,694] Trial 18 pruned. 
[I 2023-12-10 12:48:49,757] Trial 19 pruned. 
[I 2023-12-10 12:48:49,990] A new study created in memory with name: no-name-43a437de-0cb8-44d0-9948-99cd293a23f0
[I 2023-12-10 12:48:51,189] Trial 0 finished with value: 0.15735144913196564 and parameters: {'size1': 638, 'size2': 441, 'size3': 131, 'num_epochs

[I 2023-12-10 12:49:22,814] Trial 4 finished with value: 0.28677472472190857 and parameters: {'size1': 652, 'size2': 201, 'size3': 198, 'num_epochs': 172, 'lr': 0.005424817369243623}. Best is trial 0 with value: 0.1453409343957901.
[I 2023-12-10 12:49:22,830] Trial 5 pruned. 
[I 2023-12-10 12:49:22,861] Trial 6 pruned. 
[I 2023-12-10 12:49:22,886] Trial 7 pruned. 
[I 2023-12-10 12:49:22,901] Trial 8 pruned. 
[I 2023-12-10 12:49:23,024] Trial 9 pruned. 
[I 2023-12-10 12:49:23,091] Trial 10 pruned. 
[I 2023-12-10 12:49:23,155] Trial 11 pruned. 
[I 2023-12-10 12:49:23,232] Trial 12 pruned. 
[I 2023-12-10 12:49:23,300] Trial 13 pruned. 
[I 2023-12-10 12:49:23,370] Trial 14 pruned. 
[I 2023-12-10 12:49:23,434] Trial 15 pruned. 
[I 2023-12-10 12:49:23,506] Trial 16 pruned. 
[I 2023-12-10 12:49:23,579] Trial 17 pruned. 
[I 2023-12-10 12:49:23,638] Trial 18 pruned. 
[I 2023-12-10 12:49:23,704] Trial 19 pruned. 
[I 2023-12-10 12:49:24,302] A new study created in memory with name: no-name-60f09e

[I 2023-12-10 12:49:51,692] Trial 1 finished with value: 0.34721502661705017 and parameters: {'size1': 928, 'size2': 335, 'size3': 490, 'num_epochs': 192, 'lr': 0.0044332846900714375}. Best is trial 1 with value: 0.34721502661705017.
[I 2023-12-10 12:49:52,251] Trial 2 finished with value: 0.2305760681629181 and parameters: {'size1': 211, 'size2': 226, 'size3': 192, 'num_epochs': 71, 'lr': 0.008550599799481747}. Best is trial 2 with value: 0.2305760681629181.
[I 2023-12-10 12:49:52,596] Trial 3 finished with value: 0.21683569252490997 and parameters: {'size1': 371, 'size2': 157, 'size3': 389, 'num_epochs': 40, 'lr': 0.004528458910722089}. Best is trial 3 with value: 0.21683569252490997.
[I 2023-12-10 12:49:52,790] Trial 4 finished with value: 0.20456862449645996 and parameters: {'size1': 609, 'size2': 257, 'size3': 223, 'num_epochs': 17, 'lr': 0.008087920748735213}. Best is trial 4 with value: 0.20456862449645996.
[I 2023-12-10 12:49:52,819] Trial 5 pruned. 
[I 2023-12-10 12:49:52,849]

[I 2023-12-10 12:50:17,907] Trial 1 finished with value: 0.34145909547805786 and parameters: {'size1': 49, 'size2': 326, 'size3': 226, 'num_epochs': 192, 'lr': 0.009648993114502884}. Best is trial 0 with value: 0.260530948638916.
[I 2023-12-10 12:50:18,781] Trial 2 finished with value: 0.29784393310546875 and parameters: {'size1': 506, 'size2': 140, 'size3': 44, 'num_epochs': 93, 'lr': 0.00393762244639649}. Best is trial 0 with value: 0.260530948638916.
[I 2023-12-10 12:50:20,293] Trial 3 finished with value: 0.27767372131347656 and parameters: {'size1': 467, 'size2': 420, 'size3': 155, 'num_epochs': 138, 'lr': 0.0059744308562674105}. Best is trial 0 with value: 0.260530948638916.
[I 2023-12-10 12:50:22,063] Trial 4 finished with value: 0.21187551319599152 and parameters: {'size1': 615, 'size2': 223, 'size3': 105, 'num_epochs': 172, 'lr': 0.00021521415906707343}. Best is trial 4 with value: 0.21187551319599152.
[I 2023-12-10 12:50:22,095] Trial 5 pruned. 
[I 2023-12-10 12:50:22,140] Tr

...

[I 2023-12-10 12:59:27,786] Trial 4 finished with value: 0.22948221862316132 and parameters: {'size1': 407, 'size2': 196, 'size3': 66, 'num_epochs': 132, 'lr': 0.004307871723811757}. Best is trial 4 with value: 0.22948221862316132.
[I 2023-12-10 12:59:27,813] Trial 5 pruned. 
[I 2023-12-10 12:59:27,846] Trial 6 pruned. 
[I 2023-12-10 12:59:27,861] Trial 7 pruned. 
[I 2023-12-10 12:59:28,283] Trial 8 pruned. 
[I 2023-12-10 12:59:28,369] Trial 9 pruned. 
[I 2023-12-10 12:59:28,496] Trial 10 pruned. 
[I 2023-12-10 12:59:29,111] Trial 11 pruned. 
[I 2023-12-10 12:59:29,179] Trial 12 pruned. 
[I 2023-12-10 12:59:29,647] Trial 13 pruned. 
[I 2023-12-10 12:59:29,963] Trial 14 pruned. 
[I 2023-12-10 12:59:30,662] Trial 15 pruned. 
[I 2023-12-10 12:59:30,726] Trial 16 pruned. 
[I 2023-12-10 12:59:30,794] Trial 17 pruned. 
[I 2023-12-10 12:59:31,098] Trial 18 pruned. 
[I 2023-12-10 12:59:31,163] Trial 19 pruned. 
[I 2023-12-10 12:59:31,901] A new study created in memory with name: no-name-a193b9

[I 2023-12-10 13:00:02,132] Trial 4 finished with value: 0.2446451485157013 and parameters: {'size1': 599, 'size2': 37, 'size3': 412, 'num_epochs': 44, 'lr': 0.0031990518052291694}. Best is trial 2 with value: 0.23429745435714722.
[I 2023-12-10 13:00:02,361] Trial 5 finished with value: 0.248456209897995 and parameters: {'size1': 267, 'size2': 72, 'size3': 67, 'num_epochs': 27, 'lr': 0.006470583948072361}. Best is trial 2 with value: 0.23429745435714722.
[I 2023-12-10 13:00:02,390] Trial 6 pruned. 
[I 2023-12-10 13:00:02,427] Trial 7 pruned. 
[I 2023-12-10 13:00:02,448] Trial 8 pruned. 
[I 2023-12-10 13:00:02,469] Trial 9 pruned. 
[I 2023-12-10 13:00:02,540] Trial 10 pruned. 
[I 2023-12-10 13:00:02,602] Trial 11 pruned. 
[I 2023-12-10 13:00:03,821] Trial 12 finished with value: 0.23188349604606628 and parameters: {'size1': 453, 'size2': 300, 'size3': 264, 'num_epochs': 113, 'lr': 0.0029521402950234187}. Best is trial 12 with value: 0.23188349604606628.
[I 2023-12-10 13:00:03,886] Trial

[I 2023-12-10 13:00:28,448] Trial 2 finished with value: 0.22228705883026123 and parameters: {'size1': 362, 'size2': 363, 'size3': 298, 'num_epochs': 33, 'lr': 0.003823454912172719}. Best is trial 0 with value: 0.20468562841415405.
[I 2023-12-10 13:00:28,749] Trial 3 finished with value: 0.24522128701210022 and parameters: {'size1': 728, 'size2': 314, 'size3': 210, 'num_epochs': 24, 'lr': 0.0038023802332041215}. Best is trial 0 with value: 0.20468562841415405.
[I 2023-12-10 13:00:30,094] Trial 4 finished with value: 0.22911331057548523 and parameters: {'size1': 144, 'size2': 368, 'size3': 119, 'num_epochs': 156, 'lr': 0.007181512787889303}. Best is trial 0 with value: 0.20468562841415405.
[I 2023-12-10 13:00:30,183] Trial 5 pruned. 
[I 2023-12-10 13:00:30,319] Trial 6 pruned. 
[I 2023-12-10 13:00:30,336] Trial 7 pruned. 
[I 2023-12-10 13:00:30,359] Trial 8 pruned. 
[I 2023-12-10 13:00:30,382] Trial 9 pruned. 
[I 2023-12-10 13:00:30,455] Trial 10 pruned. 
[I 2023-12-10 13:00:30,512] Tri

[I 2023-12-10 13:00:52,962] Trial 1 finished with value: 0.23576924204826355 and parameters: {'size1': 451, 'size2': 507, 'size3': 34, 'num_epochs': 134, 'lr': 0.006082664802223348}. Best is trial 0 with value: 0.2307177484035492.
[I 2023-12-10 13:00:53,767] Trial 2 finished with value: 0.2143329679965973 and parameters: {'size1': 44, 'size2': 435, 'size3': 361, 'num_epochs': 64, 'lr': 0.007349610013555049}. Best is trial 2 with value: 0.2143329679965973.
[I 2023-12-10 13:00:54,614] Trial 3 finished with value: 0.20441856980323792 and parameters: {'size1': 64, 'size2': 473, 'size3': 12, 'num_epochs': 100, 'lr': 0.0009646278068881744}. Best is trial 3 with value: 0.20441856980323792.
[I 2023-12-10 13:00:55,715] Trial 4 finished with value: 0.250735878944397 and parameters: {'size1': 132, 'size2': 184, 'size3': 185, 'num_epochs': 142, 'lr': 0.001513469918657255}. Best is trial 3 with value: 0.20441856980323792.
[I 2023-12-10 13:00:55,833] Trial 5 pruned. 
[I 2023-12-10 13:00:56,000] Tria

[I 2023-12-10 13:01:18,365] Trial 1 finished with value: 0.21239641308784485 and parameters: {'size1': 665, 'size2': 379, 'size3': 59, 'num_epochs': 23, 'lr': 0.00574575983785207}. Best is trial 1 with value: 0.21239641308784485.
[I 2023-12-10 13:01:19,560] Trial 2 finished with value: 0.21868827939033508 and parameters: {'size1': 194, 'size2': 442, 'size3': 35, 'num_epochs': 136, 'lr': 0.008356490277492187}. Best is trial 1 with value: 0.21239641308784485.
[I 2023-12-10 13:01:21,242] Trial 3 finished with value: 0.23890314996242523 and parameters: {'size1': 355, 'size2': 177, 'size3': 215, 'num_epochs': 200, 'lr': 0.0083251487091948}. Best is trial 1 with value: 0.21239641308784485.
[I 2023-12-10 13:01:22,223] Trial 4 finished with value: 0.22870245575904846 and parameters: {'size1': 836, 'size2': 243, 'size3': 267, 'num_epochs': 82, 'lr': 0.0066992615209385}. Best is trial 1 with value: 0.21239641308784485.
[I 2023-12-10 13:01:22,250] Trial 5 pruned. 
[I 2023-12-10 13:01:22,267] Tria